# Final CHO Model
This notebook is to asses the validity of our reconstruction and how complete it is.

In [ ]:
# Import libraries

import gspread
import pandas as pd
import numpy as np

from cobra import Model, Reaction, Metabolite
from cobra.io import validate_sbml_model

from tqdm.notebook import tqdm

### 1. Retrieve information from the Google Sheet datasets reactions and metabolites

In [2]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns and Metabolites sheet.
rxns_sheet = cho_recon.worksheet('Rxns')
rxns_attributes_sheet = cho_recon.worksheet('Attributes')
metabolites_sheet = cho_recon.worksheet('Metabolites')

# We can extract the data using the get_all_records method and create pd DataFrames

# Reactions IDs, names, formulas, GPRs
rxns = pd.DataFrame(rxns_sheet.get_all_records())

# Reactions bounds
rxns_attributes = pd.DataFrame(rxns_attributes_sheet.get_all_records())

# Metabolites names, formulas and compartment
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())

### 2. Build a model and feed it the information from the df generated

In [3]:
# Create a model and add reactions
model = Model("iCHO")
lr = []

for _, row in rxns.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)
    
model.add_reactions(lr)
model

Name,iCHO
Memory address,12c493c70
Number of metabolites,0
Number of reactions,8196
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [4]:
# Add information to each one of the reactions
for i,r in enumerate(tqdm(model.reactions)):
    r.build_reaction_from_string(rxns['Reaction Formula'][i])
    r.name = rxns['Reaction Name'][i]
    r.subsystem = rxns['Subsystem'][i]
    r.gene_reaction_rule = str(rxns['GPR_final'][i])
    r.lower_bound = rxns_attributes['Lower bound'][i]
    r.upper_bound = rxns_attributes['Upper bound'][i]

  0%|          | 0/8196 [00:00<?, ?it/s]

unknown metabolite '10fthf5glu_c' created
unknown metabolite '10fthf5glu_l' created
unknown metabolite '10fthf5glu_m' created
unknown metabolite '10fthf6glu_c' created
unknown metabolite '10fthf6glu_l' created
unknown metabolite '10fthf6glu_m' created
unknown metabolite '10fthf7glu_c' created
unknown metabolite '10fthf7glu_l' created
unknown metabolite '10fthf7glu_m' created
unknown metabolite '10fthf_c' created
unknown metabolite '10fthf_l' created
unknown metabolite '10fthf_m' created
unknown metabolite '11docrtsl_c' created
unknown metabolite '11docrtsl_m' created
unknown metabolite '11docrtsl_r' created
unknown metabolite '11docrtstrn_c' created
unknown metabolite '11docrtstrn_m' created
unknown metabolite '11docrtstrn_r' created
unknown metabolite '12HPET_c' created
unknown metabolite 'atp_c' created
unknown metabolite 'h2o_c' created
unknown metabolite '12HPET_e' created
unknown metabolite 'adp_c' created
unknown metabolite 'h_c' created
unknown metabolite 'pi_c' created
unknown 

unknown metabolite '5mthf_e' created
unknown metabolite '5mthf_c' created
unknown metabolite '5oxpro_e' created
unknown metabolite '5oxpro_c' created
unknown metabolite '5thf_c' created
unknown metabolite '5thf_l' created
unknown metabolite '5thf_m' created
unknown metabolite '6dhf_c' created
unknown metabolite '6dhf_l' created
unknown metabolite '6dhf_m' created
unknown metabolite '6htststerone_c' created
unknown metabolite '6htststerone_e' created
unknown metabolite '6htststerone_r' created
unknown metabolite '6thf_c' created
unknown metabolite '6thf_l' created
unknown metabolite '6thf_m' created
unknown metabolite '7dhchsterol_r' created
unknown metabolite '7dhchsterol_c' created
unknown metabolite '7dhf_c' created
unknown metabolite '7dhf_l' created
unknown metabolite '7dhf_m' created
unknown metabolite '7thf_c' created
unknown metabolite '7thf_l' created
unknown metabolite '7thf_m' created
unknown metabolite 'galgluside_cho_c' created
unknown metabolite 'udpgal_c' created
unknown 

unknown metabolite 'prgstrn_c' created
unknown metabolite 'aprgstrn_c' created
unknown metabolite 'xol7ah_c' created
unknown metabolite 'xol7ah2_c' created
unknown metabolite 'xoldioloneh_c' created
unknown metabolite 'xoltriol_c' created
unknown metabolite 'xol7aone_c' created
unknown metabolite 'xoldiolone_c' created
unknown metabolite 'alaala_c' created
unknown metabolite 'alaala_e' created
unknown metabolite 'asn_L_c' created
unknown metabolite 'asn_L_e' created
unknown metabolite 'cys_L_c' created
unknown metabolite 'cys_L_e' created
unknown metabolite 'ala_D_e' created
unknown metabolite 'gln_L_c' created
unknown metabolite 'ala_D_c' created
unknown metabolite 'gln_L_e' created
unknown metabolite 'gly_c' created
unknown metabolite 'gly_e' created
unknown metabolite 'ser_L_c' created
unknown metabolite 'ser_L_e' created
unknown metabolite 'thr_L_c' created
unknown metabolite 'thr_L_e' created
unknown metabolite 'alatrna_c' created
unknown metabolite 'trnaala_c' created
unknown met

unknown metabolite 'avite1_e' created
unknown metabolite 'avite1_c' created
unknown metabolite 'avite2_e' created
unknown metabolite 'avite2_c' created
unknown metabolite 'm2mn_c' created
unknown metabolite 'man_c' created
unknown metabolite 'mn_c' created
unknown metabolite 'man_l' created
unknown metabolite 'mn_l' created
unknown metabolite 'g6p_c' created
unknown metabolite 'Asn_X_Ser/Thr_r' created
unknown metabolite 'Asn_X_Ser/Thr_l' created
unknown metabolite 'Asn_X_Ser_Thr_r' created
unknown metabolite 'Asn_X_Ser_Thr_l' created
unknown metabolite 'gbside_cho_g' created
unknown metabolite 'ga2_cho_g' created
unknown metabolite 'ga1_cho_g' created
unknown metabolite 'gm2_cho_g' created
unknown metabolite 'gm1_cho_g' created
unknown metabolite 'gd2_cho_g' created
unknown metabolite 'gd1b_cho_g' created
unknown metabolite 'gt2_cho_g' created
unknown metabolite 'gt1c_cho_g' created
unknown metabolite 'galgbside_cho_g' created
unknown metabolite 'acglcgalgluside_cho_g' created
unknown

...

unknown metabolite 'CE4988_x' created
unknown metabolite 'CE4989_c' created
unknown metabolite 'CE4988_r' created
unknown metabolite 'CE4989_r' created
unknown metabolite 'CE5945_c' created
unknown metabolite 'CE5946_c' created
unknown metabolite 'CE5945_r' created
unknown metabolite 'CE5946_r' created
unknown metabolite 'CE5947_c' created
unknown metabolite 'CE7074_c' created
unknown metabolite 'CE7072_c' created
unknown metabolite 'CE7074_r' created
unknown metabolite 'CE7072_r' created
unknown metabolite 'CE5966_m' created
unknown metabolite 'CE5967_m' created
unknown metabolite 'CE5966_x' created
unknown metabolite 'CE5967_x' created
unknown metabolite 'CE5967_c' created
unknown metabolite 'CE5968_c' created
unknown metabolite 'CE5968_m' created
unknown metabolite 'CE5968_x' created
unknown metabolite 'CE5969_m' created
unknown metabolite 'CE5970_m' created
unknown metabolite 'CE5969_x' created
unknown metabolite 'CE5970_x' created
unknown metabolite 'CE5969_c' created
unknown meta

unknown metabolite 'CN0016_r' created
unknown metabolite 'CN0017_r' created
unknown metabolite 'CN0018_c' created
unknown metabolite 'CN0018_r' created
unknown metabolite 'CN0017_x' created
unknown metabolite 'CN0018_x' created
unknown metabolite 'CN0019_c' created
unknown metabolite 'CN0019_r' created
unknown metabolite 'rna_c' created
unknown metabolite 'rna_prod_c' created
unknown metabolite 'HC02199_c' created
unknown metabolite 'HC02200_c' created
unknown metabolite 'HC02201_c' created
unknown metabolite 'xu5p_D_c' created
unknown metabolite 'Rtotal2crn_c' created
unknown metabolite 'Rtotal2crn_m' created
unknown metabolite 'Rtotal3crn_c' created
unknown metabolite 'Rtotal3crn_m' created
unknown metabolite 'Rtotalcrn_c' created
unknown metabolite 'Rtotalcrn_m' created
unknown metabolite 'sl2n2m2masn_g' created
unknown metabolite 's3lnl3fn3m2masn_g' created
unknown metabolite 's4lndl4fn4m2masn_g' created
unknown metabolite 's4lndl4n4m2masn_g' created
unknown metabolite 's4lncl4fn4m

unknown metabolite 'dhcholestanate_r' created
unknown metabolite 'dhcholestancoa_r' created
unknown metabolite 'thcholstoic_r' created
unknown metabolite 'cholcoar_r' created
unknown metabolite 'wharachd_c' created
unknown metabolite 'whtststerone_c' created
unknown metabolite 'xan_x' created
unknown metabolite 'xol7ah2_r' created
unknown metabolite 'xoldiolone_m' created
unknown metabolite 'xoltri24_c' created
unknown metabolite 'xoltri25_c' created
unknown metabolite 'xser_r' created
unknown metabolite 'xyl_D_c' created
unknown metabolite 'xylnact_D_c' created
unknown metabolite 'xylt_c' created
unknown metabolite 'zn2_c' created
unknown metabolite 'epoIm_r' created
unknown metabolite 'epo_g' created
unknown metabolite 'igg_hc_r' created
unknown metabolite 'igg_lc_r' created
unknown metabolite 'igg_g' created
unknown metabolite 'HC02119_c' created
unknown metabolite 'HC00822_l' created
unknown metabolite 'dpcoa_m' created
unknown metabolite 'HC01672_c' created
unknown metabolite 'HC0

Malformed gene_reaction_rule '__cobra_escape__100764088 or __cobra_escape__100763802 or __cobra_escape__100763802 or h728441 or h102724197 or __cobra_escape__100752303 or __cobra_escape__100758668) or ((__cobra_escape__100764088 or __cobra_escape__100763802 or __cobra_escape__100763802 or h728441 or h102724197 or __cobra_escape__100752303 or __cobra_escape__100758668) and __cobra_escape__100763802' for 100764088 or 100763802 or 100763802 or h728441 or h102724197 or 100752303 or 100758668) or ((100764088 or 100763802 or 100763802 or h728441 or h102724197 or 100752303 or 100758668) and 100763802
Traceback (most recent call last):
  File "/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/cobra/core/gene.py", line 350, in from_string
    tree = ast_parse(escaped_str, "<string>", "eval")
  File "/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<string>", line 1
    __cobra_esc

unknown metabolite 'c8crn_x' created
unknown metabolite 'HC00900_c' created
unknown metabolite 'C02470_m' created
unknown metabolite 'C02528_r' created
unknown metabolite 'tetdece1coa_m' created
unknown metabolite 'HC01231_m' created
unknown metabolite 'HC01710_c' created
unknown metabolite '3hdcoa_x' created
unknown metabolite 'xol7ah3_c' created
unknown metabolite 'HC01459_x' created
unknown metabolite '34dhmald_m' created
unknown metabolite '34dhoxmand_m' created
unknown metabolite '3mox4hpac_m' created
unknown metabolite 'homoval_m' created
unknown metabolite '3m4hpga_m' created
unknown metabolite '3mox4hoxm_m' created
unknown metabolite 'HC01652_c' created
unknown metabolite 'HC02228_c' created
unknown metabolite 'HC01842_c' created
unknown metabolite 'HC01797_c' created
unknown metabolite 'HC00004_c' created
unknown metabolite 'HC00319_c' created
unknown metabolite 'HC00319_m' created
unknown metabolite 'arachd_l' created
unknown metabolite 'hdca_l' created
unknown metabolite 'na

In [5]:
# Add information for each metabolite
metabolites_dict = metabolites.set_index('BiGG ID').to_dict('dict')
metabolites_dict['Name']

for met in model.metabolites:
    met.name = metabolites_dict['Name'][f'{met}']
    met.formula = metabolites_dict['Formula'][f'{met}']
    met.compartment = metabolites_dict['Compartment'][f'{met}'].split(' - ')[0]

apocytc_m = model.metabolites.get_by_id('apocytc_m')
apocytc_m

Metabolite identifier,apocytc_m
Name,Apocytochrome c
Memory address,0x139d30580
Formula,C8H12N4O4S2R4
Compartment,m
In 1 reaction(s),HEMELm


In [6]:
lcaa_c = model.metabolites.get_by_id('lcaa_c')
lcaa_c

Metabolite identifier,lcaa_c
Name,Long-chain acid anion
Memory address,0x13a25d780
Formula,C3H4O2R
Compartment,c
In 1 reaction(s),GapFill-R04311


In [18]:
# Check Mass Balance
for rxn in model.reactions:
    if len(rxn.check_mass_balance()) != 0:
        print(rxn.id)
        print(rxn.reaction)
        print(rxn.check_mass_balance())
        print('.......................................')

15KPROSTGF2c
nad_c + prostgf2_c <=> 15kprostgf2_c + nadh_c
{'H': -1}
.......................................
2AMACSULT
2amac_c + nadph_c + paps_c --> Lcyst_c + nadp_c + pap_c
{'O': 3, 'S': 1}
.......................................
3HAD141n7m
c142_2Z_7Zcoa_m + h2o_m <=> 3hoc141_7Zcoa_m
{'H': -4}
.......................................
3HAD260p
h2o_x + hex2coa_x <=> 3hhexcoa_x
{'H': 4}
.......................................
3HDH141n7m
3hoc141_7Zcoa_m + nad_m <=> 3oc141_7Zcoa_m + h_m + nadh_m
{'H': 4}
.......................................
4NPHSULT
4nph_c + paps_c --> 4nphsf_c + h_c + pap_c
{'O': 3, 'S': 1}
.......................................
5ADTSTSTERONESULT
5adtststerone_c + paps_c --> 5adtststerones_c + h_c + pap_c
{'O': 3, 'S': 1}
.......................................
A4GALTc
galgluside_cho_c + udpgal_c --> h_c + thcrm_cho_c + udp_c
{'F': 1, 'U': 1, 'L': 2}
.......................................
ACACT226n3m
3oc226_4Z_7Z_10Z_13Z_16Z_19Zcoa_m + coa_m --> accoa_m + tmndnccoa_m

In [19]:
# Initiliase model
for rxn_exchange in model.exchanges:
    rxn_exchange.bounds = (-1000, 1000)
bio_id = model.reactions.index('biomass_producing')
atp_id = model.reactions.index('DM_atp_c')

model.reactions[bio_id].bounds = (0, 100)
model.objective = 'biomass_producing'
# model.objective = 'DM_atp_c'
model.optimize()

,fluxes,reduced_costs
10FTHF5GLUtl,0.0,0.000000e+00
10FTHF5GLUtm,0.0,0.000000e+00
10FTHF6GLUtl,0.0,0.000000e+00
10FTHF6GLUtm,0.0,0.000000e+00
10FTHF7GLUtl,0.0,0.000000e+00
...,...,...
r2537,0.0,0.000000e+00
r2538,0.0,-1.162505e-16
r2539,0.0,0.000000e+00
ALLTTtm,0.0,1.291280e-18


In [20]:
with open('Biomass_Metabolites_Reactions.txt', 'w') as f:
    bio_mets = model.reactions.get_by_id('biomass_producing').metabolites
    for bio_met in bio_mets:
        f.write("Metabolite: " + bio_met.name + "\n")
        f.write("Reactions: " + ", ".join([r.name for r in bio_met.reactions]) + "\n")
        f.write("Formula: " + ", ".join([r.build_reaction_string() for r in bio_met.reactions]) + "\n\n")

# Detect all the excahnge reactions that have no uptake
from cobra.flux_analysis import flux_variability_analysis
FVA_Result = flux_variability_analysis(model, model.exchanges)

FVA_Result = a[a.minimum == 0]
with open('FVA_Excange.txt', 'w') as f:
    f.write("reaction\tminimum\tmaximum\n")
    for index, row in FVA_Result.iterrows():
        f.write(f"{index}\t{row['minimum']}\t{row['maximum']}\n")

/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is requir

NameError: name 'a' is not defined

In [ ]:
for reaction in model.reactions:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

10FTHF5GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF5GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF6GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF6GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF7GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF7GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHFtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHFtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSLtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSLtr blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSTRNtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSTRNtr blocked (bounds: (0, 0)), new growth rate 0.000000
12HPETATP blocked (bounds: (0, 0)), new growth rate 0.000000
12HPETUPKt blocked (bounds: (0, 0)), new growth rate 0.000000
13DAMPPOX blocked (bounds: (0, 0)), new growth rate 0.000000
15HPETATP blocked (bounds: (0, 0)), new growth rate 0.00

3HAD200p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD201n9p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD203n6m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD203n6p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n6m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n6p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD205n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD205n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD220p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD221n9p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD224n6m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD224n6p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD2251n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD2251n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD226n3m

3HDH60p blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH70m blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH80m blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH80p blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH81n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH81n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH90m blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHDMHPm blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHDMNm blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHMPm blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHPRISTp blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHTMHDp blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHTMTDp blocked (bounds: (0, 0)), new growth rate 0.000000
3HKYNAKGAT blocked (bounds: (0, 0)), new growth rate 0.000000
3HKYNAKGATm blocked (bounds: (0, 0)), new growth rate 0.000000
3HLYTCL blocked (bounds: (0, 0)), new growth rate 0.000000
3HPRODHx blocked (bounds: (0, 0)), 

AASAD3m blocked (bounds: (0, 0)), new growth rate 0.000000
AATAi blocked (bounds: (0, 0)), new growth rate 0.000000
ABC_TCDBd blocked (bounds: (0, 0)), new growth rate 0.000000
ABO1g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO2g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO3g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO4g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO5g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO6g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO7g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO8g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO9g blocked (bounds: (0, 0)), new growth rate 0.000000
ABTArm blocked (bounds: (0, 0)), new growth rate 0.000000
ABTD blocked (bounds: (0, 0)), new growth rate 0.000000
ABTti blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTD blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTDm blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTt2r blocked (bounds:

ACGALK blocked (bounds: (0, 0)), new growth rate 0.000000
ACGALK2 blocked (bounds: (0, 0)), new growth rate 0.000000
ACGALtlg blocked (bounds: (0, 0)), new growth rate 0.000000
ACGALtly blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAM2E blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAM6PSi blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMK blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMPM blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMtg blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMtl blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMtly blocked (bounds: (0, 0)), new growth rate 0.000000
ACGBGBSIDEtg blocked (bounds: (0, 0)), new growth rate 0.000000
ACGBGBSIDEtl blocked (bounds: (0, 0)), new growth rate 0.000000
ACGLUte blocked (bounds: (0, 0)), new growth rate 0.000000
ACGLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
ACGPID blocked (bounds: (0, 0)), new growth rate 0.000000
ACGSm blocked (bounds: (0, 0)), new growth r

...

MM8Ag blocked (bounds: (0, 0)), new growth rate 0.000000
MM8Ber blocked (bounds: (0, 0)), new growth rate 0.000000
MM8Cg blocked (bounds: (0, 0)), new growth rate 0.000000
MMCDm blocked (bounds: (0, 0)), new growth rate 0.000000
MMCDp blocked (bounds: (0, 0)), new growth rate 0.000000
MMCH blocked (bounds: (0, 0)), new growth rate 0.000000
MMEm blocked (bounds: (0, 0)), new growth rate 0.000000
MMMm blocked (bounds: (0, 0)), new growth rate 0.000000
MMSAD1m blocked (bounds: (0, 0)), new growth rate 0.000000
MMSAD3m blocked (bounds: (0, 0)), new growth rate 0.000000
MMSDH blocked (bounds: (0, 0)), new growth rate 0.000000
MMSDHm blocked (bounds: (0, 0)), new growth rate 0.000000
MMSOR blocked (bounds: (0, 0)), new growth rate 0.000000
MMTSADm blocked (bounds: (0, 0)), new growth rate 0.000000
MNSERGALTg blocked (bounds: (0, 0)), new growth rate 0.000000
MOD_3mhop blocked (bounds: (0, 0)), new growth rate 0.000000
MOGAT blocked (bounds: (0, 0)), new growth rate 0.000000
MSERterg blocked 

NDPK6 blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK6m blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK6n blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK7 blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK7m blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK7n blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK8 blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK8m blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK8n blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK9 blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK9m blocked (bounds: (0, 0)), new growth rate 0.000000
NDPK9n blocked (bounds: (0, 0)), new growth rate 0.000000
NH4t3r blocked (bounds: (0, 0)), new growth rate 0.000000
NH4tn blocked (bounds: (0, 0)), new growth rate 0.000000
NH4tp blocked (bounds: (0, 0)), new growth rate 0.000000
NH4tr blocked (bounds: (0, 0)), new growth rate 0.000000
NICRNTtn blocked (bounds: (0, 0)), new growth rate 0.000000
NIFEDIPINEte blocke

ORNTArm blocked (bounds: (0, 0)), new growth rate 0.000000
ORNt3m blocked (bounds: (0, 0)), new growth rate 0.000000
ORNt4m blocked (bounds: (0, 0)), new growth rate 0.000000
ORNtiDF blocked (bounds: (0, 0)), new growth rate 0.000000
OROTGLUt blocked (bounds: (0, 0)), new growth rate 0.000000
OROTi blocked (bounds: (0, 0)), new growth rate 0.000000
ORPT blocked (bounds: (0, 0)), new growth rate 0.000000
OXAACONtm blocked (bounds: (0, 0)), new growth rate 0.000000
OXAHCOtex blocked (bounds: (0, 0)), new growth rate 0.000000
OXALSUCCCL blocked (bounds: (0, 0)), new growth rate 0.000000
OXALSUCCCLm blocked (bounds: (0, 0)), new growth rate 0.000000
OXALSUCCCLp blocked (bounds: (0, 0)), new growth rate 0.000000
OXAtp blocked (bounds: (0, 0)), new growth rate 0.000000
OXOADLR blocked (bounds: (0, 0)), new growth rate 0.000000
P45011A1m blocked (bounds: (0, 0)), new growth rate 0.000000
P45011B11m blocked (bounds: (0, 0)), new growth rate 0.000000
P45011B12m blocked (bounds: (0, 0)), new gro

PE_HStg blocked (bounds: (0, 0)), new growth rate 0.000000
PE_HStm blocked (bounds: (0, 0)), new growth rate 0.000000
PEt blocked (bounds: (0, 0)), new growth rate 0.000000
PFK blocked (bounds: (0, 0)), new growth rate 0.000000
PFK26 blocked (bounds: (0, 0)), new growth rate 0.000000
PFK_3 blocked (bounds: (0, 0)), new growth rate 0.000000
PFK_4 blocked (bounds: (0, 0)), new growth rate 0.000000
PGAM1_PhosHydro blocked (bounds: (0, 0)), new growth rate 0.000000
PGCD blocked (bounds: (0, 0)), new growth rate 0.000000
PGDI blocked (bounds: (0, 0)), new growth rate 0.000000
PGDIr blocked (bounds: (0, 0)), new growth rate 0.000000
PGESc blocked (bounds: (0, 0)), new growth rate 0.000000
PGESr blocked (bounds: (0, 0)), new growth rate 0.000000
PGI blocked (bounds: (0, 0)), new growth rate 0.000000
PGISr blocked (bounds: (0, 0)), new growth rate 0.000000
PGK blocked (bounds: (0, 0)), new growth rate 0.000000
PGL blocked (bounds: (0, 0)), new growth rate 0.000000
PGLYCABCte blocked (bounds: (

POMGNT2g blocked (bounds: (0, 0)), new growth rate 0.000000
POMGNTr blocked (bounds: (0, 0)), new growth rate 0.000000
POMKr blocked (bounds: (0, 0)), new growth rate 0.000000
POXYLTg blocked (bounds: (0, 0)), new growth rate 0.000000
PPA blocked (bounds: (0, 0)), new growth rate 0.000000
PPA2 blocked (bounds: (0, 0)), new growth rate 0.000000
PPA2m blocked (bounds: (0, 0)), new growth rate 0.000000
PPADCOAL blocked (bounds: (0, 0)), new growth rate 0.000000
PPADCOALm blocked (bounds: (0, 0)), new growth rate 0.000000
PPAP blocked (bounds: (0, 0)), new growth rate 0.000000
PPAer blocked (bounds: (0, 0)), new growth rate 0.000000
PPAm blocked (bounds: (0, 0)), new growth rate 0.000000
PPAn blocked (bounds: (0, 0)), new growth rate 0.000000
PPAt blocked (bounds: (0, 0)), new growth rate 0.000000
PPAtm blocked (bounds: (0, 0)), new growth rate 0.000000
PPBNGS blocked (bounds: (0, 0)), new growth rate 0.000000
PPCDC blocked (bounds: (0, 0)), new growth rate 0.000000
PPCOACm blocked (bounds

In [ ]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(model, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)